## Segmenting and Clustering Neighborhoods in Toronto
#### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto./
####           1. Start by creating a new Notebook for this assignment.
####           2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of                   postal codes and to transform the data into a pandas dataframe.

In [1]:
%pip install beautifulsoup4 # Installing the BeautifulSoup4
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
import requests # library to handle requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Installing the Conda
!conda update -n base -c defaults conda --yes 

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
#from IPython.display import Image 
#from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
#from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda-navigator==1.9.2=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::curl==7.61.0=h84994c4_0
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::h5py==2.8.0=py37h989c5e5_3
  - defaults/linux-64::imageio==2.4.1=py37_0
  - defaults/linux-64::jupyter==1.0.0=py37_7
  - defaults/linux-64::libcurl==7.61.0=h1ad7b7a_0
  - defaults/linux-64::libssh2==1.8.0=h9cfc8f7_4
  - defaults/linux-64::matplotlib==2.2.3=py37hb69df0a_0
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::mkl_fft==1.0.4=py37h4414c95_1
  - defau

In [3]:
%pip install requests
import requests
# Loading data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# Using beautiful soup to parse the HTML/XML codes.
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_wikipedia_page,'html')
# print(soup.prettify())

Note: you may need to restart the kernel to use updated packages.


In [4]:
My_table = soup.find('table', {"class": 'wikitable sortable'})

In [7]:
New_Table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# Extracting a clean form of the table
for tr_cell in New_Table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)


In [9]:
Unique_Postal = set(Postcode)
print('num of unique Postal codes:', len(Unique_Postal))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in Unique_Postal:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


In [10]:
Toronto_Dictionary = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_Toronto = pd.DataFrame.from_dict(Toronto_Dictionary)
df_Toronto.to_csv('Toronto_Part-A.csv')
df_Toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9L,North York,Humber Summit
4,M4Y,Downtown Toronto,Church and Wellesley
5,M9N,York,Weston
6,M3J,North York,"Northwood Park, York University"
7,M2H,North York,Hillcrest Village
8,M2J,North York,Henry Farm
9,M5S,Downtown Toronto,University of Toronto


In [11]:
df_Toronto.shape

(77, 3)